<a href="https://colab.research.google.com/github/KevinSBSon/Vehicle-Proximity-Warning-System/blob/main/Car_Detection_and_Warning_Tensorflow.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Vehicle Proximity Warning System

###Import Packages

In [ ]:
import cv2
import glob
import argparse
import os
import matplotlib.pyplot as plt
from matplotlib.pyplot import imshow
import scipy.io
import scipy.misc
import numpy as np
import pandas as pd
import PIL
from PIL import ImageFont, ImageDraw, Image
import tensorflow as tf
from tensorflow.python.framework.ops import EagerTensor
from tensorflow.keras.models import load_model
from yad2k.models.keras_yolo import yolo_head
from yad2k.utils.utils import draw_boxes, get_colors_for_classes, scale_boxes, read_classes, read_anchors, preprocess_image
%matplotlib inline

###Implement YOLO filter boxes

In [ ]:
#Filter boxes by a threshold value

def yolo_filter_boxes(boxes, box_confidence, box_class_probs, threshold = .6):
    
    #Compute box scores
    box_scores = box_confidence*box_class_probs

    #Find the box_classes using the max box_scores
    box_classes = tf.math.argmax(box_scores, axis = -1)
    box_class_scores = tf.math.reduce_max(box_scores, axis = -1)
    
    #Create a filtering mask 
    filtering_mask = (box_class_scores >= threshold)
    
    #Apply the mask
    scores = tf.boolean_mask(box_class_scores,filtering_mask)
    boxes = tf.boolean_mask(boxes,filtering_mask)
    classes = tf.boolean_mask(box_classes,filtering_mask)
    
    return scores, boxes, classes

###Implement IoU

In [ ]:
# Implement the Intersection over Union between two boxes

def iou(box1, box2):
    
    (box1_x1, box1_y1, box1_x2, box1_y2) = box1
    (box2_x1, box2_y1, box2_x2, box2_y2) = box2

    xi1 = max(box1_x1, box2_x1)
    yi1 = max(box1_y1, box2_y1)
    xi2 = min(box1_x2, box2_x2)
    yi2 = min(box1_y2, box2_y2)
    inter_width = max(xi2-xi1, 0)
    inter_height =  max(yi2-yi1, 0)
    inter_area = inter_width * inter_height
    
    # Calculate the Union area 
    box1_area = (box1_x2-box1_x1)*(box1_y2-box1_y1)
    box2_area = (box2_x2-box2_x1)*(box2_y2-box2_y1)
    union_area = box1_area + box2_area - inter_area
    
    # IoU
    iou = inter_area/union_area
    
    return iou

###Implement YOLO Non-max suppression

In [ ]:
# Implement YOLO Non-max suppression 

def yolo_non_max_suppression(scores, boxes, classes, max_boxes = 10, iou_threshold = 0.5):
        
    max_boxes_tensor = tf.Variable(max_boxes, dtype='int32')    

    # Get the list of indices corresponding to boxes
    nms_indices = tf.image.non_max_suppression(boxes, scores, max_boxes, iou_threshold = iou_threshold)
    
    # Select only nms_indices from scores, boxes and classes
    scores = tf.gather(scores, nms_indices)
    boxes = tf.gather(boxes, nms_indices)
    classes = tf.gather(classes, nms_indices)
    
    return scores, boxes, classes

###Process YOLO outputs

In [ ]:
# Convert YOLO predictions to the corners of the box
def yolo_boxes_to_corners(box_xy, box_wh):
    box_mins = box_xy - (box_wh / 2.)
    box_maxes = box_xy + (box_wh / 2.)

    return tf.keras.backend.concatenate([
        box_mins[..., 1:2],  
        box_mins[..., 0:1],  
        box_maxes[..., 1:2],  
        box_maxes[..., 0:1]  
    ])

In [ ]:
# Converts the output of YOLO to predicted boxes
def yolo_eval(yolo_outputs, image_shape = (720, 1280), max_boxes=10, score_threshold=.6, iou_threshold=.5):
    
    # Retrieve outputs of the YOLO model
    box_xy, box_wh, box_confidence, box_class_probs = yolo_outputs
    
    # Convert boxes box_xy and box_wh to corner coordinates
    boxes = yolo_boxes_to_corners(box_xy, box_wh)
    
    # Filter boxes by a threshold value
    scores, boxes, classes = yolo_filter_boxes(boxes, box_confidence, box_class_probs, threshold = score_threshold)
    
    # Scale boxes back to original image shape
    boxes = scale_boxes(boxes, image_shape)
    
    # Perform Non-max suppression
    scores, boxes, classes = yolo_non_max_suppression(scores, boxes, classes, max_boxes = max_boxes, iou_threshold = iou_threshold)
    
    return scores, boxes, classes

###Load pre-trained YOLO model

In [ ]:
# Read classes and anchors
class_names = read_classes("model_data/coco_classes.txt") #A class('Warning') for objects that are close is added
anchors = read_anchors("model_data/yolo_anchors.txt") #A color for Warning is added
model_image_size = (608, 608)

In [ ]:
# Load model
yolo_model = load_model("model_data/", compile=False)

In [ ]:
# YOLO model summary
yolo_model.summary()

Model: "functional_1"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 608, 608, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv2d (Conv2D)                (None, 608, 608, 32  864         ['input_1[0][0]']                
                                )                                                                 
                                                                                                  
 batch_normalization (BatchNorm  (None, 608, 608, 32  128        ['conv2d[0][0]']                 
 alization)                     )                                                      

                                                                                                  
 batch_normalization_8 (BatchNo  (None, 38, 38, 512)  2048       ['conv2d_8[0][0]']               
 rmalization)                                                                                     
                                                                                                  
 leaky_re_lu_8 (LeakyReLU)      (None, 38, 38, 512)  0           ['batch_normalization_8[0][0]']  
                                                                                                  
 conv2d_9 (Conv2D)              (None, 38, 38, 256)  131072      ['leaky_re_lu_8[0][0]']          
                                                                                                  
 batch_normalization_9 (BatchNo  (None, 38, 38, 256)  1024       ['conv2d_9[0][0]']               
 rmalization)                                                                                     
          

 conv2d_20 (Conv2D)             (None, 38, 38, 64)   32768       ['leaky_re_lu_12[0][0]']         
                                                                                                  
 leaky_re_lu_18 (LeakyReLU)     (None, 19, 19, 1024  0           ['batch_normalization_18[0][0]'] 
                                )                                                                 
                                                                                                  
 batch_normalization_20 (BatchN  (None, 38, 38, 64)  256         ['conv2d_20[0][0]']              
 ormalization)                                                                                    
                                                                                                  
 conv2d_19 (Conv2D)             (None, 19, 19, 1024  9437184     ['leaky_re_lu_18[0][0]']         
                                )                                                                 
          

###Detect objects using YOLO

In [ ]:
#Predict boxes from the image
def predict(image, image_file = ''):

    # Preprocess the image
    model_image_size = (608, 608)
    resized_image = image.resize(tuple(reversed(model_image_size)), Image.BICUBIC)
    image_data = np.array(resized_image, dtype='float32')
    image_data /= 255.
    image_data = np.expand_dims(image_data, 0)
    
    # Run through YOLO model
    yolo_model_outputs = yolo_model(image_data)
    yolo_outputs = yolo_head(yolo_model_outputs, anchors, len(class_names))
    out_scores, out_boxes, out_classes = yolo_eval(yolo_outputs, [image.size[1],  image.size[0]], 10, 0.3, 0.5)

    # Draw bounding boxes on the image
    processed_image = draw_boxes(image, out_boxes, out_classes, class_names2, out_scores) #draw_boxes function is modified

    return processed_image



###Run Code1: for real-time detection

In [ ]:
# Code for real time detection

cap = cv2.VideoCapture(0)

while True:
    ret, frame = cap.read()
    if ret == False:
        break
    im = Image.fromarray(frame, 'RGB')
    im = predict(im)
    cv2.imshow('Vehicle Detection System', im)
    if cv2.waitKey(1) == ord('q'):
        break    

out.release()
cap.release()
cv2.destroyAllWindows()

###Run Code2: for a recorded video

In [ ]:
# Code for a recorded video

cap = cv2.VideoCapture('Track_clip4.mp4')

#List to store images
img_array = []
i = 0

while(cap.isOpened()):
    ret, frame = cap.read()
    if ret == False:
        break
    im = Image.fromarray(frame, 'RGB')
    im = predict(im)
    height, width, layers = frame.shape
    size = (width, height)
    if i == 0:
        out = cv2.VideoWriter('project3.MP4', cv2.VideoWriter_fourcc(*'MP4V'), 60, size)
        i+=1
    img_array.append(im)
    out.write(img_array[i])    
    
out.release()
cap.release()

cv2.destroyAllWindows()